Get help from: https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22



In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd
downloaded = drive.CreateFile({'id':'1eZAsnldRlvKeM21dp2Ie35HWzeU38aNF'}) ## Load Secondary_Source.csv in the edge_list_creation folder
downloaded.GetContentFile('bit_table_warren_top.json')  
file_dictionary = pd.read_json('bit_table_warren_top.json')

In [4]:
!pip install pytorch-pretrained-bert
from pytorch_pretrained_bert import BertTokenizer, BertModel

     |████████████████████████████████| 133kB 3.4MB/s 


In [0]:
file_dictionary = file_dictionary.sort_values(["Length"])[["Description", "Opinion"]]

In [0]:
import torch

In [0]:
data = file_dictionary.values

In [0]:
data = data[:150]

In [0]:
content = [v for i, v in enumerate(data)]

In [0]:
def get_ready(sentence):
  tokens=tokenizer.tokenize(sentence)
  tokens = ['[CLS]'] + tokens + ['[SEP]']
  T=100
  padded_tokens=tokens +['[PAD]' for _ in range(T-len(tokens))]
  attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens]
  seg_ids=[0 for _ in range(len(padded_tokens))]
  padded_tokens = padded_tokens[:150]
  attn_mask = attn_mask[:150]
  seg_ids = seg_ids[:150]
  sent_ids=tokenizer.convert_tokens_to_ids(padded_tokens)
  token_ids = torch.tensor(sent_ids).unsqueeze(0) 
  attn_mask = torch.tensor(attn_mask).unsqueeze(0) 
  seg_ids = torch.tensor(seg_ids).unsqueeze(0)
  return token_ids, attn_mask, seg_ids

In [11]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

100%|██████████| 407873900/407873900 [00:05<00:00, 80435161.19B/s]


In [12]:
dic = {}
for i in range(len(content)):
  name = content[i][0]
  print("working on doc number "+ str(i))
  token_ids, attn_mask, seg_ids= get_ready(content[i][1])
  hidden_reps, cls_head = bert_model(token_ids, attention_mask = attn_mask,token_type_ids = seg_ids)
  vec = cls_head
  dic[name] = vec

working on doc number 0
working on doc number 1
working on doc number 2
working on doc number 3
working on doc number 4
working on doc number 5
working on doc number 6
working on doc number 7
working on doc number 8
working on doc number 9
working on doc number 10
working on doc number 11
working on doc number 12
working on doc number 13
working on doc number 14
working on doc number 15
working on doc number 16
working on doc number 17
working on doc number 18
working on doc number 19
working on doc number 20
working on doc number 21
working on doc number 22
working on doc number 23
working on doc number 24
working on doc number 25
working on doc number 26
working on doc number 27
working on doc number 28
working on doc number 29
working on doc number 30
working on doc number 31
working on doc number 32
working on doc number 33
working on doc number 34
working on doc number 35
working on doc number 36
working on doc number 37
working on doc number 38
working on doc number 39
working on

In [0]:
from scipy import spatial
def similar(a, b):
  a = a.detach().numpy()
  b = b.detach().numpy()
  result = 1 - spatial.distance.cosine(a, b)
  return result

In [0]:
threshold = 0.9 
Source = []
Target = []
Weight = []

In [15]:
for i in range(150):
  for k in range(i+1, 150):
    print("working on " + str(i) + " comparing "+ str(k))
    name_one = list(dic)[i]
    name_two = list(dic)[k]
    score = similar(dic[name_one],dic[name_two])
    if score > threshold:
      Source.append(name_one)
      Target.append(name_two)
      Weight.append(score) 

Streaming output truncated to the last 5000 lines.
working on 49 comparing 101
working on 49 comparing 102
working on 49 comparing 103
working on 49 comparing 104
working on 49 comparing 105
working on 49 comparing 106
working on 49 comparing 107
working on 49 comparing 108
working on 49 comparing 109
working on 49 comparing 110
working on 49 comparing 111
working on 49 comparing 112
working on 49 comparing 113
working on 49 comparing 114
working on 49 comparing 115
working on 49 comparing 116
working on 49 comparing 117
working on 49 comparing 118
working on 49 comparing 119
working on 49 comparing 120
working on 49 comparing 121
working on 49 comparing 122
working on 49 comparing 123
working on 49 comparing 124
working on 49 comparing 125
working on 49 comparing 126
working on 49 comparing 127
working on 49 comparing 128
working on 49 comparing 129
working on 49 comparing 130
working on 49 comparing 131
working on 49 comparing 132
working on 49 comparing 133
working on 49 comparing 1

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Weight':Weight})

In [17]:
result

,Source,Target,Weight
0,"OPPENHEIMER v. CALIFORNIA., 378 U.S. 929 (1964)","AHLSTEDT v. US, 377 U.S. 873 (1963)",0.988061
1,"OPPENHEIMER v. CALIFORNIA., 378 U.S. 929 (1964)","DONATO v. US, 375 U.S. 871 (1962)",0.974045
2,"OPPENHEIMER v. CALIFORNIA., 378 U.S. 929 (1964)","LITTLE v. OREGON, 385 U.S. 902 (1966)",0.985382
3,"OPPENHEIMER v. CALIFORNIA., 378 U.S. 929 (1964)","GARTNER, v. OVERHOLSER, 345 U.S. 973 (1953)",0.989530
4,"OPPENHEIMER v. CALIFORNIA., 378 U.S. 929 (1964)","MISSIOURI v. NEBRASKA., 390 U.S. 993 (1968)",0.973861
...,...,...,...
10737,"AMERICAN LEGION POST NO. 51 v. PENNSYLVANIA, 3...",TACOMA ASSOCIATION OF CREDIT MEN v. WASHINGTON...,0.984442
10738,"AMERICAN LEGION POST NO. 51 v. PENNSYLVANIA, 3...","ROBERTOY v. MICHIGAN, 364 U.S. 519 (1960)",0.951625
10739,"DAYTON RUBBER v. CORDOVAN ASSOC., 364 U.S. 299...",TACOMA ASSOCIATION OF CREDIT MEN v. WASHINGTON...,0.959129
10740,"DAYTON RUBBER v. CORDOVAN ASSOC., 364 U.S. 299...","ROBERTOY v. MICHIGAN, 364 U.S. 519 (1960)",0.960348


In [18]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
result.to_csv('BERT_edge_list2.csv', index=False)
!cp BERT_edge_list2.csv /content/drive/My\ Drive/19-20/Spring/Data88/Supreme_Court/BERT